# Picocoder
_A Raspberry Pi Pico microcode glitcher_

## Connection
We connect with the Pi Pico and setup some glitching options.

In [1]:
%matplotlib widget

POWERSUPPLY_PORT = '/dev/ttyACM0'
GLITCHER_PORT = '/dev/ttyACM1'
GLITCHER_BAUD = 115200

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from importlib import reload # for debugging

import glitch_utils
from glitch_utils import GlitchyMcGlitchFace, GlitchResult
from power_supply import PowerSupply, KA3305P
glitch_utils = reload(glitch_utils)

try:
	del ps
	del glitcher
	del gc
except:
	pass
ps = KA3305P(port=POWERSUPPLY_PORT, cycle_wait=0.5)
ps.con()
ps.power_cycle()

glitcher = glitch_utils.GlitchyMcGlitchFace(GLITCHER_PORT, GLITCHER_BAUD)
if not glitcher.ping():
	raise Exception("Glitcher not responding")
if not glitcher.ping_target():
	raise Exception("Target not responding")

def reset_target(ps: PowerSupply, glitcher: GlitchyMcGlitchFace) -> None:
	ps.power_cycle()
	if not glitcher.ping_target():
		raise Exception("Target not responding after reset")

## Estimating glitch external offset
We obtain an estimate (in us) of the wait time between trigger and glitch.

In [2]:
ext_offset = glitcher.estimate_ext_offset()
if ext_offset < 0:
	raise Exception(f'Invalid offset {ext_offset}')
print(f"Estimated offset: {ext_offset}")

Estimated offset: 183


## Measure UART delay
This command is used to measure (externally) the time it takes for the Pico to
detect data on the UART RX pin and to start processing it. <br>
The debug pin (GPIO 16, shown blue) is toggled when the Pico detects data.

![Oscilloscope capture](../firmware/inject_v2/img/fast_response_time_busy_wait_on_UART_with_timeout.png)

In [4]:
glitcher.uart_toggle_debug_pin()

## Voltage range

Find where the CPU starts to stall. We ask the target to send 50 bytes to the
pico, and then we bring the voltage down for a fairly long time.

The target will then either behave normally or die altogether, not just stall
for some clock cycles. This test is fairly slow as it involves a lot of
external resets, but within a handful of minutes a sensible working voltage
can be identified right above the reset cluster

![Voltage range test plot (ignore the squares)](img/voltage_range_plot.png)

Here, for example, we can see that down to 59 (0b111011) the target is mostly
working, but there is a significant amount of crashed at 58 (0b111010). 59 will
then be the starting value for the glitching range.

### Oscilloscope captures
| Color | Description |
| --- | --- |
| Yellow | Target UART TX |
| Pink | PMBus/I2C SCL |
| Blue | PMBus/I2C SDA |
| Green | VCore |

#### No effect
When the voltage drop does not affect the target, it will successfully send the
50 bytes.

![Safe voltage and width](img/voltage_range_alive.png)

#### CPU halt
When the voltage is too low, the target will hang and not finish sending data.

![Too low voltage](img/voltage_range_dead.png)

In [ ]:
# NOTE This figure must be generated in a different cell than the one that calls the plot function
#	   Otherwise the plot will not be displayed until the cell is done executing (no live plot)
fig, ax = plt.subplots()
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

In [ ]:
EXPECTED_BYTES = 50

gc_volt = glitch_utils.GlitchController(groups=[r.name for r in GlitchResult], parameters=['width', 'voltage'])
gc_volt.set_range('width', 1, 100)
gc_volt.set_range('voltage', 0b0110011, 0b1001011) # 1V-1.24V - See Table 6-3 in TPS65094 datasheet

glitcher.ext_offset = 500 # Offset fixed

for glitch_setting in gc_volt.rand_glitch_values():
	try:
		result, data = glitcher.find_crash_voltage(glitch_setting, EXPECTED_BYTES)

		ax.plot(*glitch_setting, result)
		fig.canvas.draw() # Guarantees live update of the plot whenever a new point is added

		if result in [GlitchResult.DEAD, GlitchResult.RESET, GlitchResult.BROKEN]:
			reset_target(ps, glitcher)
	except KeyboardInterrupt:
		fig.canvas.draw()
		break # Gentle stop, otherwise the plot might look empty

## Estimate Vp in other ways
Misuse glitch to estimate Vp

In [ ]:
gc_vp = glitch_utils.GlitchController(groups=[r.name for r in GlitchResult], parameters=['ext_offset', 'prep_voltage'])
gc_vp.set_range('ext_offset', ext_offset+20, ext_offset+50)
# gc_vp.set_range('prep_voltage', 0b0101001, 0b0110011) # 0.9V-1V - See Table 6-3 in TPS65094 datasheet
gc_vp.set_range('prep_voltage', 0b0011111, 0b0101001) # 0.8V-0.9V - See Table 6-3 in TPS65094 datasheet
# gc_vp.set_range('prep_voltage', 0b0010101, 0b0011111) # 0.7V-0.8V - See Table 6-3 in TPS65094 datasheet
glitcher.voltage = 0b0111101 # 1.1V

# NOTE This figure must be generated in a different cell than the one that calls the plot function
#	   Otherwise the plot will not be displayed until the cell is done executing (no live plot)
fig, ax = plt.subplots()
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

In [ ]:
prev_ext_offset = 0
prev_prep_voltage = 0

for glitch_setting in gc_vp.rand_glitch_values():
	try:
		[ext_offset, prep_voltage] = glitch_setting
		# glitcher.voltage = prep_voltage # Use same as prep during "glitch"
		read_result, read_data = glitcher.mul_test_vp(glitch_setting)
		gc_vp.add_result(glitch_setting, read_result)
		if read_result == GlitchResult.SUCCESS:
			[performed, result_a, result_b] = read_data
			print(f'SUCCESS! With {performed} iterations, {result_a} != {result_b}')
		if read_result == GlitchResult.WEIRD:
			print(f'Got weird response: 0x{read_data.hex()}')

		if read_result == GlitchResult.BROKEN and prev_ext_offset != 0 and prev_prep_voltage != 0:
			# Actually something from previous iteration broke it
			print(f'prev_ext_offset = {prev_ext_offset}, prev_prep_voltage = {prev_prep_voltage}')
			ax.plot(prev_ext_offset, prev_prep_voltage, read_result)
		else:
			# Print current point
			ax.plot(ext_offset, prep_voltage, read_result)

		fig.canvas.draw() # Guarantees live update of the plot whenever a new point is added

		if read_result in [GlitchResult.DEAD, GlitchResult.RESET, GlitchResult.BROKEN]:
			reset_target(ps, glitcher)
		prev_ext_offset = ext_offset
		prev_prep_voltage = prep_voltage
		break
	except KeyboardInterrupt:
		fig.canvas.draw()
		break # Gentle stop, otherwise the plot might look empty

## Glitch loop

In [ ]:
gc = glitch_utils.GlitchController(groups=[r.name for r in GlitchResult], parameters=['ext_offset', 'width', 'voltage'])
gc.set_range('ext_offset', ext_offset-40, ext_offset)
gc.set_range('width', 1, 10)
gc.set_range('voltage', 0b0010101, 0b0101001) # 0.7V-0.9V - See Table 6-3 in TPS65094 datasheet
glitcher.prep_voltage = 0b0111010 # 1.07V

# NOTE This figure must be generated in a different cell than the one that calls the plot function
#	   Otherwise the plot will not be displayed until the cell is done executing (no live plot)
fig, ax = plt.subplots()
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

In [ ]:
for glitch_setting in gc.rand_glitch_values():
	try:
		read_result, read_data = glitcher.glitch_mul(glitch_setting)
		gc.add_result(glitch_setting, read_result)
		if read_result == GlitchResult.SUCCESS:
			[performed, result_a, result_b] = read_data
			print(f'SUCCESS! With {performed} iterations, {result_a} != {result_b}')
		if read_result == GlitchResult.WEIRD:
			print(f'Got weird response: 0x{read_data.hex()}')

		[_, width, voltage] = glitch_setting
		ax.plot(width, voltage, read_result)
		fig.canvas.draw() # Guarantees live update of the plot whenever a new point is added

		if read_result in [GlitchResult.DEAD, GlitchResult.RESET, GlitchResult.BROKEN]:
			reset_target(ps, glitcher)
		else:
			break
	except KeyboardInterrupt:
		fig.canvas.draw()
		break # Gentle stop, otherwise the plot might look empty